# Install

In [ ]:
!git clone https://github.com/farzane-maghsoudi/HiSGAN
import os
os.chdir('HiSGAN/')
!pip install -r requirements.txt

In [ ]:
!pip install involution_pytorch
!pip install thop
!pip install --upgrade git+https://github.com/rwightman/pytorch-image-models.git
!pip install timm

#dataset

In [ ]:
#!bash ./download_dataset.sh mini
!bash ./dataset/download_dataset.sh vangogh2photo

#Training

In [ ]:
!python main.py --dataset vangogh2photo --light True

Restoring from the previous checkpoint

In [ ]:
!python main.py --dataset vangogh2photo --resume True --light True

#Testing

In [ ]:
!python main.py --dataset mini --phase test --light True

#Copy checkpoint to GoogleDrive and convert




In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
!mkdir ./results

In [ ]:
#b = "/content/gdrive/MyDrive/checkpointTTL-GAN/horse2zebra_params_latest.pt"
#c = "/content/TTL-GAN/results/horse2zebra_params_latest.pt"
f1 = "/content/gdrive/MyDrive/checkpointQS-GAN/vangogh2photo_params_0035000.pt"
#f2 = "/content/TTL-GAN/results/horse2zebra/model/vangogh2photo_params_0035000.pt"
cc = "/content/HiSGAN/results/vangogh2photo_params_latest.pt"

Colab to Drive

In [ ]:
import shutil
shutil.copy( c , b )
shutil.copy( f2 , f1 )

Drive to Colab

In [ ]:
import shutil
shutil.copy( f1 , cc )

'/content/QS-GAN/results/vangogh2photo_params_latest.pt'

#Test save

In [ ]:
#import shutil
#b = "/content/gdrive/MyDrive/results/501/UGATIT_light_apple2orange_lsgan_4resblock_6dis_1_1_10_10_1000_1_1_1_sn_smoothing"
#c = "/content/UG/results/UGATIT_light_apple2orange_lsgan_4resblock_6dis_1_1_10_10_1000_1_1_1_sn_smoothing"
!cp "/content/UG/results/UGATIT_light_apple2orange_lsgan_4resblock_6dis_1_1_10_10_1000_1_1_1_sn_smoothing" -r "/content/gdrive/MyDrive/results"

In [ ]:
n_blocks = 8
for i in range(2, n_blocks+1):
  if i%3 == 0:
    x2 = 3
    print (i, x2)
  if i%3 == 1:
    x3 = 4
    print (i, x3)
  if i%3 == 2:
    if i> 4:
      x4 = 10
      print (i, x4)
    else:
      x4 = 5
      print (i, x4)

In [ ]:
import torch
import torch.nn as nn
a = torch.randn(1,256, 64, 64)
#a = torch.reshape(a, (256, 64, 64))
atten  = torch.nn.MultiheadAttention(64, 8)
relu = torch.nn.ReLU(True)
class MultiSelfAttentionBlock(nn.Module):
    def __init__(self, atten, relu, dim =64, featur = 256):
      super(MultiSelfAttentionBlock, self).__init__()
      self.dim = dim
      self.featur = featur
      self.atten  = atten
      self.relu  = relu


    def forward(self, x):
      print("as")
      out = torch.reshape(x, (self.featur, self.dim, self.dim))
      out, _ = self.atten(out, out, out)
      out = self.relu(torch.reshape(out, (1, self.featur, self.dim, self.dim)))
      print("a")

      return out

#out, _ = atten(a, a, a)
#out = torch.reshape(out, (1, 256, 64, 64))
for i in range(3):
  out = MultiSelfAttentionBlock(a, atten, relu)
  #print (out.shape)

In [ ]:
import torch
import torch.nn as nn
a = torch.randn(1,256, 64, 64)
atten  = torch.nn.MultiheadAttention(64, 8)
relu = torch.nn.ReLU(True)
for i in range(3):
  print("as")
  out = torch.reshape(a, (256, 64, 64))
  out, _ = atten(out, out, out)
  out = torch.reshape(out, (1, 256, 64, 64))
  out = relu(out)
  print("a")
  print (out.shape)

In [ ]:
import torch
import torch.nn as nn
x = torch.randn(1,256, 64, 64)
atten  = torch.nn.MultiheadAttention(64, 8)
relu = torch.nn.ReLU(True)
n_blocks = 8
out = torch.reshape(x, (256, 64, 64))
out, _ = atten(out, out, out)
out = relu(torch.reshape(out, (1, 256, 64, 64)))
xa1 = out
xa11 = out
if n_blocks>1:
  for i in range(2, 8+1):
    if i%3 == 2:
      out = torch.reshape(out, (256, 64, 64))
      out, _ = atten(out, out, out)
      xa2 = out = relu(torch.reshape(out, (1, 256, 64, 64)))
      print (i)
    elif i%3 == 0:
      out = torch.reshape(out + xa1, (256, 64, 64))
      out, _ = atten(out, out, out)
      xa3 = out = relu(torch.reshape(out, (1, 256, 64, 64)))
      print (i)
    elif i < 6:
      out = torch.reshape(out + xa1 + xa2, (256, 64, 64))
      out, _ = atten(out, out, out)
      xa1 = out = relu(torch.reshape(out, (1, 256, 64, 64)))
      print (i)
    else:
      out = torch.reshape(out + xa1 + xa2 + xa11, (256, 64, 64))
      out, _ = atten(out, out, out)
      xa11 = xa1
      xa1 = out = relu(torch.reshape(out, (1, 256, 64, 64)))
      print (i)

In [ ]:
import torch
import torch.nn as nn
a = torch.randn(1,128, 56, 56)
b = torch.randn(1,128, 28, 28)
c = torch.randn(1,128, 14, 14)
up1 = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
up2 = nn.Upsample(scale_factor=4, mode='bilinear', align_corners=True)
pad1 = nn.ReflectionPad2d(4)
pad2 = nn.ReflectionPad2d(4)
pad3 = nn.ReflectionPad2d(4)
a = pad1 (a)
b = up1(b)
b = pad2(b)
c = up2(c)
c = pad3(c)
print (a.shape)
print(b.shape)
print(c.shape)

In [ ]:
self.save(self.checkpoint_dir, counter)
				    a = (epoch+1)*1000
				    b = "/content/gdrive/MyDrive/checkpoint/UGATIT_light_apple2orange_lsgan_4resblock_6dis_1_1_10_10_1000_1_1_1_sn_smoothing"
				    c = "/content/UG/checkpoint/UGATIT_light_apple2orange_lsgan_4resblock_6dis_1_1_10_10_1000_1_1_1_sn_smoothing"
				    f1 = "/UGATIT_light.model-"+ str(a)+ ".index"
				    f2 = "/UGATIT_light.model-"+ str(a)+ ".meta"
				    f3 = "/UGATIT_light.model-" + str(a)+ ".data-00000-of-00001"
				    f = "/checkpoint"
				    import shutil
				    shutil.copy( c+f1 , b+f1 )
				    shutil.copy( c+f2 , b+f2 )
				    shutil.copy( c+f3 , b+f3 )
				    shutil.copy( c+f , b+f )

In [ ]:
import torch
import torch.nn as nn
input = torch.randn(1,256, 256,3)
S = torch.linalg.svdvals(input)
#out = torch.reshape(s1[0,:,0], (-1,))
R = torch.reshape(S[0,:,0], (-1,))
G = torch.reshape(S[0,:,1], (-1,))
B = torch.reshape(S[0,:,2], (-1,))

c1 = torch.reshape(S[0,:,0], (-1,))*torch.reshape(S[0,:,1], (-1,))
c2 = torch.cartesian_prod(torch.reshape(S[0,:,0], (-1,)), torch.reshape(S[0,:,2], (-1,)))
c3 = torch.cartesian_prod(torch.reshape(S[0,:,1], (-1,)), torch.reshape(S[0,:,2], (-1,)))
c4 = torch.cartesian_prod(torch.reshape(S[0,:,0], (-1,)), torch.reshape(S[0,:,1], (-1,)), torch.reshape(S[0,:,2], (-1,)))
R1 = torch.cartesian_prod(R, G)
print(c1.shape)
print(c2.shape)
print(c3.shape)
print(c4.shape)
print(R.shape)
print(G.shape)
print(B.shape)
print(R1.shape)

In [ ]:
import torch.nn as nn
import torch
input = torch.randn(1,3, 256,256)
S = torch.linalg.svdvals(input)  #(1,3,256)
K = 64

c1 = torch.matmul(torch.reshape(S[0,0,0:K], (K,1)), torch.reshape(S[0,1,0:K], (1,K)))
c2 = torch.matmul(torch.reshape(S[0,1,0:K], (K,1)), torch.reshape(S[0,2,0:K], (1,K)))
c3 = torch.matmul(torch.reshape(S[0,0,0:K], (K,1)), torch.reshape(S[0,2,0:K], (1,K)))
c4 = torch.matmul(torch.reshape(c1, (K, K, 1)), torch.reshape(S[0,2,0:K], (1, 1, K)))

svd = torch.cat((torch.reshape(c1, (1,1,K,K)),torch.reshape(c2, (1,1,K,K)),torch.reshape(c3, (1,1,K,K))), dim=1)
print(svd.shape)
svd = torch.cat((svd,torch.reshape(c4, (1,K,K,K))), dim=1)
print(svd.shape)

torch.Size([1, 3, 64, 64])
torch.Size([1, 67, 64, 64])


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

class SELayer(nn.Module):
    def __init__(self, channel, reduction=16):
        super(SELayer, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(channel, channel // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channel // reduction, channel, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        res = x * y.expand_as(x)
        return res

class SeparableFourierUnit(nn.Module):

    def __init__(self, in_channels, out_channels, groups=1, kernel_size=3):
        # bn_layer not used
        super(SeparableFourierUnit, self).__init__()
        self.groups = groups
        row_out_channels = out_channels // 2
        col_out_channels = out_channels - row_out_channels
        self.row_conv = torch.nn.Conv2d(in_channels=in_channels * 2,
                                        out_channels=row_out_channels * 2,
                                        kernel_size=(kernel_size, 1),
                                        # kernel size is always like this, but the data will be transposed
                                        stride=1, padding=(kernel_size // 2, 0),
                                        padding_mode='reflect',
                                        groups=self.groups, bias=False)
        self.col_conv = torch.nn.Conv2d(in_channels=in_channels * 2,
                                        out_channels=col_out_channels * 2,
                                        kernel_size=(kernel_size, 1),
                                        # kernel size is always like this, but the data will be transposed
                                        stride=1, padding=(kernel_size // 2, 0),
                                        padding_mode='reflect',
                                        groups=self.groups, bias=False)
        self.row_bn = torch.nn.BatchNorm2d(row_out_channels * 2)
        self.col_bn = torch.nn.BatchNorm2d(col_out_channels * 2)
        self.relu = torch.nn.ReLU(inplace=True)

    def process_branch(self, x, conv, bn):
        batch = x.shape[0]

        r_size = x.size()
        # (batch, c, h, w/2+1, 2)
        ffted = torch.fft.rfft(x, norm="ortho")
        ffted = torch.stack((ffted.real, ffted.imag), dim=-1)
        ffted = ffted.permute(0, 1, 4, 2, 3).contiguous()  # (batch, c, 2, h, w/2+1)
        ffted = ffted.view((batch, -1,) + ffted.size()[3:])

        ffted = self.relu(bn(conv(ffted)))

        ffted = ffted.view((batch, -1, 2,) + ffted.size()[2:]).permute(
            0, 1, 3, 4, 2).contiguous()  # (batch,c, t, h, w/2+1, 2)
        ffted = torch.complex(ffted[..., 0], ffted[..., 1])

        output = torch.fft.irfft(ffted, s=x.shape[-1:], norm="ortho")
        return output

    def forward(self, x):
        rowwise = self.process_branch(x, self.row_conv, self.row_bn)
        colwise = self.process_branch(x.permute(0, 1, 3, 2), self.col_conv, self.col_bn).permute(0, 1, 3, 2)
        out = torch.cat((rowwise, colwise), dim=1)
        return out

class FourierUnit(nn.Module):

    def __init__(self, in_channels, out_channels, groups=1, spatial_scale_factor=None, spatial_scale_mode='bilinear',
                 spectral_pos_encoding=False, use_se=False, se_kwargs=None, ffc3d=False, fft_norm='ortho'):
        # bn_layer not used
        super(FourierUnit, self).__init__()
        self.groups = groups

        self.conv_layer = torch.nn.Conv2d(in_channels=in_channels * 2 + (2 if spectral_pos_encoding else 0),
                                          out_channels=out_channels * 2,
                                          kernel_size=1, stride=1, padding=0, groups=self.groups, bias=False)
        self.bn = torch.nn.BatchNorm2d(out_channels * 2)
        self.relu = torch.nn.ReLU(inplace=True)

        # squeeze and excitation block
        self.use_se = use_se
        if use_se:
            if se_kwargs is None:
                se_kwargs = {}
            self.se = SELayer(self.conv_layer.in_channels, **se_kwargs)

        self.spatial_scale_factor = spatial_scale_factor
        self.spatial_scale_mode = spatial_scale_mode
        self.spectral_pos_encoding = spectral_pos_encoding
        self.ffc3d = ffc3d
        self.fft_norm = fft_norm

    def forward(self, x):
        batch = x.shape[0]

        if self.spatial_scale_factor is not None:
            orig_size = x.shape[-2:]
            x = F.interpolate(x, scale_factor=self.spatial_scale_factor, mode=self.spatial_scale_mode,
                              align_corners=False)

        r_size = x.size()
        # (batch, c, h, w/2+1, 2)
        fft_dim = (-3, -2, -1) if self.ffc3d else (-2, -1)
        ffted = torch.fft.rfftn(x, dim=fft_dim, norm=self.fft_norm)
        ffted = torch.stack((ffted.real, ffted.imag), dim=-1)
        ffted = ffted.permute(0, 1, 4, 2, 3).contiguous()  # (batch, c, 2, h, w/2+1)
        ffted = ffted.view((batch, -1,) + ffted.size()[3:])

        if self.spectral_pos_encoding:
            height, width = ffted.shape[-2:]
            coords_vert = torch.linspace(0, 1, height)[None, None, :, None].expand(batch, 1, height, width).to(ffted)
            coords_hor = torch.linspace(0, 1, width)[None, None, None, :].expand(batch, 1, height, width).to(ffted)
            ffted = torch.cat((coords_vert, coords_hor, ffted), dim=1)

        if self.use_se:
            ffted = self.se(ffted)

        ffted = self.conv_layer(ffted)  # (batch, c*2, h, w/2+1)
        ffted = self.relu(self.bn(ffted))

        ffted = ffted.view((batch, -1, 2,) + ffted.size()[2:]).permute(
            0, 1, 3, 4, 2).contiguous()  # (batch,c, t, h, w/2+1, 2)
        ffted = torch.complex(ffted[..., 0], ffted[..., 1])

        ifft_shape_slice = x.shape[-3:] if self.ffc3d else x.shape[-2:]
        output = torch.fft.irfftn(ffted, s=ifft_shape_slice, dim=fft_dim, norm=self.fft_norm)

        if self.spatial_scale_factor is not None:
            output = F.interpolate(output, size=orig_size, mode=self.spatial_scale_mode, align_corners=False)

        return output


class FFCBlock(nn.Module):

    def __init__(self, in_channels=256, out_channels=256, stride=1, groups=1, enable_lfu=True, separable_fu=False, **fu_kwargs):
        # bn_layer not used
        super(FFCBlock, self).__init__()
        self.enable_lfu = enable_lfu
        if stride == 2:
            self.downsample = nn.AvgPool2d(kernel_size=(2, 2), stride=2)
        else:
            self.downsample = nn.Identity()

        self.stride = stride
        conv1 = [nn.utils.spectral_norm(nn.Conv2d(in_channels, out_channels//2,
                                                  kernel_size=1, groups=groups, bias=False)),nn.GELU()]

        fu_class = SeparableFourierUnit if separable_fu else FourierUnit
        self.fu = fu_class(
            out_channels // 2, out_channels // 2, groups, **fu_kwargs)
        if self.enable_lfu:
            self.lfu = fu_class(
                out_channels // 2, out_channels // 2, groups)
        self.conv2 = nn.Conv2d(out_channels // 2, out_channels, kernel_size=1, groups=groups, bias=False)
        self.conv1 = nn.Sequential(*conv1)

    def forward(self, x):
        x = self.downsample(x)
        x = self.conv1(x)
        output = self.fu(x)

        if self.enable_lfu:
            n, c, h, w = x.shape
            split_no = 2
            split_s = h // split_no
            xs = torch.cat(torch.split(
                x[:, :c // 4], split_s, dim=-2), dim=1).contiguous()
            xs = torch.cat(torch.split(xs, split_s, dim=-1),
                           dim=1).contiguous()
            xs = self.lfu(xs)
            xs = xs.repeat(1, 1, split_no, split_no).contiguous()
        else:
            xs = 0

        output = self.conv2(x + output + xs)
        print(output.shape)

        return output


input = torch.randn(1, 256, 64, 64)
ST = FFCBlock(in_channels=256, out_channels=256)
x1 = ST(input)


torch.Size([1, 256, 64, 64])


In [ ]:
pip install involution_pytorch

In [ ]:
import torch
from involution_pytorch import Inv2d
import torch.nn as nn
inv = Inv2d(
    channels=16,
    kernel_size=3,
    stride=1
)

x = torch.rand(1, 16, 32, 32)
#y = inv(x) # [1, 16, 32, 32]
#print (y.shape)
m = nn.Conv2d(16, 33, 3, stride=2)
output = m(x)
print (output.shape)

In [ ]:
!pip install einops

In [ ]:
import torch
import torch.nn as nn
from einops import rearrange
import torch.utils.checkpoint as checkpoint
from timm.models.layers import DropPath, to_2tuple, trunc_normal_
import math

class WindowAttention(nn.Module):
    r""" Window based multi-head self attention (W-MSA) module with relative position bias.
    It supports both of shifted and non-shifted window.

    Args:
        dim (int): Number of input channels.
        window_size (tuple[int]): The height and width of the window.
        num_heads (int): Number of attention heads.
        qkv_bias (bool, optional):  If True, add a learnable bias to query, key, value. Default: True
        qk_scale (float | None, optional): Override default qk scale of head_dim ** -0.5 if set
        attn_drop (float, optional): Dropout ratio of attention weight. Default: 0.0
        proj_drop (float, optional): Dropout ratio of output. Default: 0.0
    """

    def __init__(self, dim, window_size, num_heads, qkv_bias=True, qk_scale=None, attn_drop=0., proj_drop=0.):

        super().__init__()
        self.dim = dim
        self.window_size = window_size  # Wh, Ww
        self.num_heads = num_heads
        head_dim = dim // num_heads
        self.scale = qk_scale or head_dim ** -0.5

        # define a parameter table of relative position bias
        self.relative_position_bias_table = nn.Parameter(
            torch.zeros((2 * window_size[0] - 1) * (2 * window_size[1] - 1), num_heads))  # 2*Wh-1 * 2*Ww-1, nH

        # get pair-wise relative position index for each token inside the window
        coords_h = torch.arange(self.window_size[0])
        coords_w = torch.arange(self.window_size[1])
        coords = torch.stack(torch.meshgrid([coords_h, coords_w]))  # 2, Wh, Ww
        coords_flatten = torch.flatten(coords, 1)  # 2, Wh*Ww
        relative_coords = coords_flatten[:, :, None] - coords_flatten[:, None, :]  # 2, Wh*Ww, Wh*Ww
        relative_coords = relative_coords.permute(1, 2, 0).contiguous()  # Wh*Ww, Wh*Ww, 2
        relative_coords[:, :, 0] += self.window_size[0] - 1  # shift to start from 0
        relative_coords[:, :, 1] += self.window_size[1] - 1
        relative_coords[:, :, 0] *= 2 * self.window_size[1] - 1
        relative_position_index = relative_coords.sum(-1)  # Wh*Ww, Wh*Ww
        self.register_buffer("relative_position_index", relative_position_index)

        self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)

        trunc_normal_(self.relative_position_bias_table, std=.02)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x, mask=None):
        """
        Args:
            x: input features with shape of (num_windows*B, N, C)
            mask: (0/-inf) mask with shape of (num_windows, Wh*Ww, Wh*Ww) or None
        """
        B_, N, C = x.shape
        print("att","B_",B_)
        print("att","N",N)
        print("att","C",C)
        qkv = self.qkv(x).reshape(B_, N, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
        q, k, v = qkv[0], qkv[1], qkv[2]  # make torchscript happy (cannot use tensor as tuple)
        print("att","q",q.shape)
        print("att","k",k.shape)
        print("att","v",v.shape)

        q = q * self.scale
        attn = (q @ k.transpose(-2, -1))
        print("att","attn",attn.shape)

        relative_position_bias = self.relative_position_bias_table[self.relative_position_index.view(-1)].view(
            self.window_size[0] * self.window_size[1], self.window_size[0] * self.window_size[1], -1)  # Wh*Ww,Wh*Ww,nH
        relative_position_bias = relative_position_bias.permute(2, 0, 1).contiguous()  # nH, Wh*Ww, Wh*Ww
        attn = attn + relative_position_bias.unsqueeze(0)

        if mask is not None:
            nW = mask.shape[0]
            attn = attn.view(B_ // nW, nW, self.num_heads, N, N) + mask.unsqueeze(1).unsqueeze(0)
            attn = attn.view(-1, self.num_heads, N, N)
            attn = self.softmax(attn)
        else:
            attn = self.softmax(attn)

        attn = self.attn_drop(attn)

        x = (attn @ v).transpose(1, 2).reshape(B_, N, C)
        print("att","x",x.shape)
        x = self.proj(x)
        x = self.proj_drop(x)
        print("att","xout",x.shape)
        return x

    def extra_repr(self) -> str:
        return f'dim={self.dim}, window_size={self.window_size}, num_heads={self.num_heads}'

    def flops(self, N):
        # calculate flops for 1 window with token length of N
        flops = 0
        # qkv = self.qkv(x)
        flops += N * self.dim * 3 * self.dim
        # attn = (q @ k.transpose(-2, -1))
        flops += self.num_heads * N * (self.dim // self.num_heads) * N
        #  x = (attn @ v)
        flops += self.num_heads * N * N * (self.dim // self.num_heads)
        # x = self.proj(x)
        flops += N * self.dim * self.dim
        return flops

def window_partition(x, window_size):
    """
    Args:
        x: (B, H, W, C)
        window_size (int): window size

    Returns:
        windows: (num_windows*B, window_size, window_size, C)
    """
    B, H, W, C = x.shape
    print("window_partition","B",B)
    print("window_partition","H",H)
    print("window_partition","H",H)
    print("window_partition","C",C)

    x = x.view(B, H // window_size, window_size, W // window_size, window_size, C)
    windows = x.permute(0, 1, 3, 2, 4, 5).contiguous().view(-1, window_size, window_size, C)
    print("window_partition","windows",windows.shape)
    print("window_partition","x",x.shape)
    return windows


def window_reverse(windows, window_size, H, W):
    """
    Args:
        windows: (num_windows*B, window_size, window_size, C)
        window_size (int): Window size
        H (int): Height of image
        W (int): Width of image

    Returns:
        x: (B, H, W, C)
    """
    print("window_reverse","windows",windows.shape)
    print("window_reverse","window_size",window_size)
    #print("window_reverse","B",B)
    B = int(windows.shape[0] / (H * W / window_size / window_size))
    print("window_reverse","B",B)
    x = windows.view(B, H // window_size, W // window_size, window_size, window_size, -1)
    x = x.permute(0, 1, 3, 2, 4, 5).contiguous().view(B, H, W, -1)
    print("window_reverse","x",x.shape)
    return x

x = torch.rand(1,256,64,64)
window_size=2
shift_size=4
mlp_ratio=4.
qkv_bias=True
qk_scale=None
drop=0.
attn_drop=0.
drop_path=0.
act_layer=nn.GELU
H, W = (64,64)
attn = WindowAttention(
            256, window_size=to_2tuple(window_size), num_heads=8,
            qkv_bias=qkv_bias, qk_scale=qk_scale, attn_drop=attn_drop, proj_drop=drop)
if shift_size > 0:
  # calculate attention mask for SW-MSA
  img_mask = torch.zeros((1, H, W, 1))  # 1 H W 1
  h_slices = (slice(0, -window_size),
              slice(-window_size, -shift_size),
              slice(-shift_size, None))
  w_slices = (slice(0, -window_size),
              slice(-window_size, -shift_size),
              slice(-shift_size, None))
  cnt = 0
  for h in h_slices:
      for w in w_slices:
          img_mask[:, h, w, :] = cnt
          cnt += 1

  mask_windows = window_partition(img_mask, window_size)  # nW, window_size, window_size, 1
  mask_windows = mask_windows.view(-1, window_size * window_size)
  attn_mask = mask_windows.unsqueeze(1) - mask_windows.unsqueeze(2)
  attn_mask = attn_mask.masked_fill(attn_mask != 0, float(-100.0)).masked_fill(attn_mask == 0, float(0.0))
else:
  attn_mask = None

print("attn_mask",attn_mask.shape)

print("x",x.shape)
x = x.flatten(2).transpose(1, 2)

print("x",x.shape)
H, W = (64,64)
B, L, C = x.shape
print("B",B)
print("L",L)
print("C",C)
assert L == H * W, "input feature has wrong size"
print("L",L)
print("x",x.shape)
shortcut = x
x = x.view(B, H, W, C)
print("x",x.shape)
# cyclic shift
if shift_size > 0:
  shifted_x = torch.roll(x, shifts=(-shift_size, -shift_size), dims=(1, 2))
else:
  shifted_x = x

# partition windows
print("shifted_x", shifted_x.shape)
x_windows = window_partition(shifted_x, window_size)  # nW*B, window_size, window_size, C
print("x_windows", x_windows.shape)
x_windows = x_windows.view(-1, window_size * window_size, C)  # nW*B, window_size*window_size, C
print("x_windows",x_windows.shape)
# W-MSA/SW-MSA
print("x",x.shape)
attn_windows = attn(x_windows, mask=attn_mask)  # nW*B, window_size*window_size, C
print("attn_windows",attn_windows.shape)
print("x",x.shape)
# merge windows
attn_windows = attn_windows.view(-1, window_size, window_size, C)
shifted_x = window_reverse(attn_windows, window_size, H, W)  # B H' W' C

# reverse cyclic shift
print("x",x.shape)
if shift_size > 0:
  x = torch.roll(shifted_x, shifts=(shift_size, shift_size), dims=(1, 2))
else:
  x = shifted_x
print("x",x.shape)
x = x.view(B, H * W, C)
print("x",x.shape)

In [ ]:
#block1 = Block(dim=embed_dims[i], mask=True if (j%2==1 and i<num_stages-1) else False, num_heads=num_heads[i],
#                mlp_ratio=mlp_ratios[i], qkv_bias=qkv_bias, qk_scale=qk_scale,
#                drop=drop_rate, attn_drop=attn_drop_rate, drop_path=dpr[cur + j], norm_layer=norm_layer,
#                sr_ratio=sr_ratios[i], linear=linear)
#block1 = Block(dim=512, mask=False, num_heads=2, mlp_ratio=4, qkv_bias=False, qk_scale=None,
#            drop=0., attn_drop=0., drop_path=0.,norm_layer=nn.LayerNorm, sr_ratio=8, linear=False)

# print()

#block1 = Block(dim=16, mask=False, num_heads=8, mlp_ratio=4, qkv_bias=False, qk_scale=None,
#                drop=0., attn_drop=0., drop_path=0.,norm_layer=nn.LayerNorm, sr_ratio=8, linear=False)

#x = torch.rand(1, 16, 32, 32)
#_, _, H, W = x.shape
#mask= None
#x, mask = block1(x, H, W, mask)

In [ ]:
from pickle import FALSE
import torch
import torch.nn as nn
import torch.nn.functional as F
from functools import partial

from timm.models.layers import DropPath, trunc_normal_
import math

def local_conv(dim):
    return nn.Conv2d(dim, dim, kernel_size=3, padding=1, stride=1, groups=dim)

class Attention(nn.Module):
    def __init__(self, dim, mask, num_heads=8, qkv_bias=False, qk_scale=None, attn_drop=0., proj_drop=0., sr_ratio=1, linear=False):
        super().__init__()
        assert dim % num_heads == 0, f"dim {dim} should be divided by num_heads {num_heads}."

        self.dim = dim
        self.num_heads = num_heads
        head_dim = dim // num_heads
        self.scale = qk_scale or head_dim ** -0.5
        self.sr_ratio=sr_ratio
        if sr_ratio>1:
            if mask:
                self.q = nn.Linear(dim, dim, bias=qkv_bias)
                self.kv1 = nn.Linear(dim, dim, bias=qkv_bias)
                self.kv2 = nn.Linear(dim, dim, bias=qkv_bias)

                if self.sr_ratio==8:
                    #r1, r2, r3 = 32,64,4  # for h = 64
                    r1, r2, r3 = 16, 8, 4
                elif self.sr_ratio==4:
                    r1, r2, r3 = 8, 4, 2
                elif self.sr_ratio==2:
                    r1, r2, r3 = 2, 1, None
                self.f1 = nn.Linear(r1, 1)
                self.f2 = nn.Linear(r2, 1)
                if r3 is not None:
                    self.f3 = nn.Linear(r3, 1)
            else:
                self.sr = nn.Conv2d(dim, dim, kernel_size=sr_ratio, stride=sr_ratio)
                self.norm = nn.LayerNorm(dim)
                self.act = nn.GELU()

                self.q1 = nn.Linear(dim, dim//2, bias=qkv_bias)
                self.kv1 = nn.Linear(dim, dim, bias=qkv_bias)
                self.q2 = nn.Linear(dim, dim // 2, bias=qkv_bias)
                self.kv2 = nn.Linear(dim, dim, bias=qkv_bias)
        else:
            self.q = nn.Linear(dim, dim, bias=qkv_bias)
            self.kv = nn.Linear(dim, dim * 2, bias=qkv_bias)

        self.lepe_linear = nn.Linear(dim, dim)
        self.lepe_conv = local_conv(dim)
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)

        self.linear = linear
        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=.02)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)
        elif isinstance(m, nn.Conv2d):
            fan_out = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
            fan_out //= m.groups
            m.weight.data.normal_(0, math.sqrt(2.0 / fan_out))
            if m.bias is not None:
                m.bias.data.zero_()

    def forward(self, x, H, W, mask):
        x = x.flatten(2).transpose(1, 2)
        B, L, C = x.shape
        B_, N= H, W
        lepe = self.lepe_conv(self.lepe_linear(x).transpose(1, 2).view(B, C, H, W)).view(H, C, -1).transpose(-1, -2)  #(1,H*W,C)
        x = x.view(B, H, W, C).reshape(H, W, C)

        if self.sr_ratio > 1:
            if mask is None:
                # global
                q1 = self.q1(x).reshape(B_, N, self.num_heads//2, C // self.num_heads).permute(0, 2, 1, 3)
                x_ = x.permute(2, 0, 1).reshape(B, C, H, W)
                x_1 = self.sr(x_).reshape(C, B_//self.sr_ratio,N//self.sr_ratio).permute(1,2,0)
                x_1 = self.act(self.norm(x_1))
                kv1 = self.kv1(x_1).reshape(B_//self.sr_ratio, -1, 2, self.num_heads//2, C // self.num_heads).permute(2, 0, 3, 1, 4)
                k1, v1 = kv1[0].repeat(self.sr_ratio,1,self.sr_ratio,1), kv1[1].repeat(self.sr_ratio,1,self.sr_ratio,1) #B_ head N C

                attn1 = (q1 @ k1.transpose(-2, -1)) * self.scale #B head Nq Nkv
                attn1 = attn1.softmax(dim=-1)
                attn1 = self.attn_drop(attn1)
                x1 = (attn1 @ v1).transpose(1, 2).reshape(B_, N, C//2)

                global_mask_value = torch.mean(attn1.detach().mean(1), dim=1) # B Nk  #max ?  mean ?

                # local
                q2 = self.q2(x).reshape(B_, N, self.num_heads // 2, C // self.num_heads).permute(0, 2, 1, 3) #B head N C
                kv2 = self.kv2(x_.reshape(B_, C, -1).permute(0, 2, 1)).reshape(B_, -1, 2, self.num_heads // 2,
                                                                          C // self.num_heads).permute(2, 0, 3, 1, 4)
                k2, v2 = kv2[0], kv2[1]
                q_window = 8
                window_size= 8
                q2, k2, v2 = window_partition(q2, q_window, H, W), window_partition(k2, window_size, H, W), \
                             window_partition(v2, window_size, H, W)
                attn2 = (q2 @ k2.transpose(-2, -1)) * self.scale
                # (B*numheads*num_windows, window_size*window_size, window_size*window_size)
                attn2 = attn2.softmax(dim=-1)
                attn2 = self.attn_drop(attn2)

                x2 = (attn2 @ v2)  # B*numheads*num_windows, window_size*window_size, C   .transpose(1, 2).reshape(B, N, C)
                x2 = window_reverse(x2, q_window, H, W, self.num_heads // 2).transpose(1, 2).reshape(B_, N, C//2)

                local_mask_value = torch.mean(attn2.detach().view(B, self.num_heads//2, H*W//window_size, window_size, window_size).mean(1), dim=2)
                local_mask_value = local_mask_value.view(B, H // window_size, W // window_size, window_size, window_size)
                local_mask_value=local_mask_value.permute(0, 1, 3, 2, 4).contiguous().view(B, H, W)

                # mask B H W
                x = torch.cat([x1, x2], dim=-1)
                x = self.proj(x+lepe)
                x = self.proj_drop(x)
                # cal mask
                mask = local_mask_value+global_mask_value
                mask_1 = mask.view(1, H * W)
                mask_2 = mask.permute(0, 2, 1).reshape(1, H * W)
                mask = [mask_1, mask_2]
            else:
                q = self.q(x).reshape(B_, N, self.num_heads, C // self.num_heads).permute(0, 2, 1, 3)

                # mask [local_mask global_mask]  local_mask [value index]  value [B, H, W]
                # use mask to fuse
                mask_1, mask_2 = mask
                mask_sort1, mask_sort_index1 = torch.sort(mask_1, dim=1)
                mask_sort2, mask_sort_index2 = torch.sort(mask_2, dim=1)
                if self.sr_ratio == 8:
                    r1, r2, r3 = 16, 8, 4
                    token1, token2, token3 = H * W // (r1*4), H * W // (r2*2), H * W //(r3*4)
                elif self.sr_ratio==4:
                    r1, r2, r3 = 8, 4, 2
                    token1, token2, token3 = H * W // (r1*4), H * W // (r2*2), H * W //(r3*4)
                elif self.sr_ratio == 2:
                    token1, token2 = H * W // (r1*2), H * W // (r2*1)

                if self.sr_ratio==4 or self.sr_ratio==8:
                    x_ = x.permute( 1, 0, 2).reshape(1, H * W, C)
                    x = x.reshape(1, H * W, C)
                    p1 = torch.gather(x, 1, mask_sort_index1[:, :H * W // 4].unsqueeze(-1).repeat(1, 1, C))  # B, N//4, C
                    p2 = torch.gather(x, 1, mask_sort_index1[:, H * W // 4:H * W // 4 * 3].unsqueeze(-1).repeat(1, 1, C))
                    p3 = torch.gather(x, 1, mask_sort_index1[:, H * W // 4 * 3:].unsqueeze(-1).repeat(1, 1, C))
                    seq1 = torch.cat([self.f1(p1.permute(0, 2, 1).reshape(B, C, token1, -1)).squeeze(-1),
                                      self.f2(p2.permute(0, 2, 1).reshape(B, C, token2, -1)).squeeze(-1),
                                      self.f3(p3.permute(0, 2, 1).reshape(B, C, token3, -1)).squeeze(-1)], dim=-1).permute(0,2,1)  # B N C
                    p1_ = torch.gather(x_, 1, mask_sort_index2[:, :H * W // 4].unsqueeze(-1).repeat(1, 1, C))  # B, N//4, C
                    p2_ = torch.gather(x_, 1, mask_sort_index2[:, H * W // 4:H * W // 4 * 3].unsqueeze(-1).repeat(1, 1, C))
                    p3_ = torch.gather(x_, 1, mask_sort_index2[:, H * W // 4 * 3:].unsqueeze(-1).repeat(1, 1, C))
                    seq2 = torch.cat([self.f1(p1_.permute(0, 2, 1).reshape(B, C, token1, -1)).squeeze(-1),
                                      self.f2(p2_.permute(0, 2, 1).reshape(B, C, token2, -1)).squeeze(-1),
                                      self.f3(p3_.permute(0, 2, 1).reshape(B, C, token3, -1)).squeeze(-1)], dim=-1).permute(0,2,1)  # B N C
                elif self.sr_ratio==2:
                    x_ = x.permute( 1, 0, 2).reshape(1, H * W, C)
                    x = x.reshape(1, H * W, C)

                    p1 = torch.gather(x, 1, mask_sort_index1[:, :H * W // 2].unsqueeze(-1).repeat(1, 1, C))  # B, N//2, C
                    p2 = torch.gather(x, 1, mask_sort_index1[:, H * W // 2:].unsqueeze(-1).repeat(1, 1, C))

                    seq1 = torch.cat([self.f1(p1.permute(0, 2, 1).reshape(B, C, token1, -1)).squeeze(-1),
                                      self.f2(p2.permute(0, 2, 1).reshape(B, C, token2, -1)).squeeze(-1)], dim=-1).permute(0, 2, 1)  # B N C

                    p1_ = torch.gather(x_, 1, mask_sort_index2[:, :H * W // 2].unsqueeze(-1).repeat(1, 1, C))  # B, N//4, C
                    p2_ = torch.gather(x_, 1, mask_sort_index2[:, H * W // 2:].unsqueeze(-1).repeat(1, 1, C))
                    print("p1_",p1_.shape)
                    print("p2_",p2_.shape)

                    seq2 = torch.cat([self.f1(p1_.permute(0, 2, 1).reshape(B, C, token1, -1)).squeeze(-1),
                                      self.f2(p2_.permute(0, 2, 1).reshape(B, C, token2, -1)).squeeze(-1)], dim=-1).permute(0, 2, 1)  # B N C

                kv1 = self.kv1(seq1).reshape(B_, -1, 2, self.num_heads // 2, C // self.num_heads).permute(2, 0, 3, 1, 4) # kv B heads N C
                kv2 = self.kv2(seq2).reshape(B_, -1, 2, self.num_heads // 2, C // self.num_heads).permute(2, 0, 3, 1, 4)
                kv = torch.cat([kv1, kv2], dim=2)

                k, v = kv[0], kv[1]
                print("seq1", seq1.shape)
                print("seq2", seq2.shape)
                print("k", k.shape)
                print("q", q.shape)
                attn = (q @ k.transpose(-2, -1)) * self.scale
                attn = attn.softmax(dim=-1)
                attn = self.attn_drop(attn)

                x = (attn @ v).transpose(1, 2).reshape(B_, N, C)
                x = self.proj(x+lepe)
                x = self.proj_drop(x)
                mask=None

        else:
            q = self.q(x).reshape(B_, N, self.num_heads, C // self.num_heads).permute(0, 2, 1, 3)
            kv = self.kv(x).reshape(B_, -1, 2, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)

            k, v = kv[0], kv[1]

            attn = (q @ k.transpose(-2, -1)) * self.scale
            attn = attn.softmax(dim=-1)
            attn = self.attn_drop(attn)
            x = (attn @ v).transpose(1, 2).reshape(B_, N, C)
            x = self.proj(x+lepe)
            x = self.proj_drop(x)

            mask=None
        return x, mask

def window_partition(x, window_size, H, W):
    """
    Args:
        x: (B, H, W, C)
        window_size (int): window size

    Returns:
        windows: (num_windows*B, window_size, window_size, C)
    """
    B_, num_heads, N, C = x.shape
    B = 1
    x = x.contiguous().view(B_*num_heads, N, C).contiguous().view(B*num_heads, H, W, C)
    B, H, W, C = x.shape
    x = x.view(B, H // window_size, window_size, W // window_size, window_size, C)
    windows = x.permute(0, 1, 3, 2, 4, 5).contiguous().view( -1,window_size, window_size, C)
    return windows  #(B*numheads*num_windows, window_size, window_size, C)


def window_reverse(windows, window_size, H, W, head):
    Bhead = int(windows.shape[0] / (H * W / window_size / window_size))
    x = windows.view(Bhead, H // window_size, W // window_size, window_size, window_size, -1)
    x = x.permute(0, 1, 3, 2, 4, 5).contiguous().view(Bhead, H, W, -1).view(Bhead//head, head, H, W, -1)\
        .contiguous().permute(0,2,3,1,4).contiguous().view(Bhead//head, H, W, -1).view(Bhead//head, H*W, -1)
    return x #(B, H, W, C)

x = torch.randn( 1,256,64,64)
B, C, H, W = x.shape
mask= None
attn = Attention(
        dim=C, mask=True,
        num_heads=8, qkv_bias=True, qk_scale=False,
        attn_drop=0., proj_drop=0, sr_ratio=4, linear=False)
outx, mask = attn(x, H, W, mask)

print(outx.shape) #[64, 64, 256]


In [ ]:
import torch.nn as nn
import torch
from torch.nn import functional as F
import torch.nn.functional as fnn
from torch.autograd import Variable
import numpy as np
from torchvision import models

class Vgg19(torch.nn.Module):
    def __init__(self, requires_grad=False):
        super(Vgg19, self).__init__()
        vgg_pretrained_features = models.vgg19(pretrained=True).features
        self.slice1 = torch.nn.Sequential()
        self.slice2 = torch.nn.Sequential()
        self.slice3 = torch.nn.Sequential()
        self.slice4 = torch.nn.Sequential()
        self.slice5 = torch.nn.Sequential()
        for x in range(2):
            self.slice1.add_module(str(x), vgg_pretrained_features[x])
        for x in range(2, 7):
            self.slice2.add_module(str(x), vgg_pretrained_features[x])
        for x in range(7, 12):
            self.slice3.add_module(str(x), vgg_pretrained_features[x])
        for x in range(12, 21):
            self.slice4.add_module(str(x), vgg_pretrained_features[x])
        for x in range(21, 30):
            self.slice5.add_module(str(x), vgg_pretrained_features[x])
        if not requires_grad:
            for param in self.parameters():
                param.requires_grad = False

    def forward(self, X):
        h_relu1 = self.slice1(X)
        h_relu2 = self.slice2(h_relu1)
        h_relu3 = self.slice3(h_relu2)
        h_relu4 = self.slice4(h_relu3)
        h_relu5 = self.slice5(h_relu4)
        return [h_relu1, h_relu2, h_relu3, h_relu4, h_relu5]

vgg = Vgg19().cuda()
l1 = nn.L1Loss()
ab = False
a = torch.rand(1, 3, 224, 224)
p = torch.rand(1, 3, 224, 224)
n = torch.rand(1, 3, 224, 224)
a_vgg, p_vgg, n_vgg = vgg(a), vgg(p), vgg(n)
loss = 0
print(loss)

d_ap, d_an = 0, 0
for i in range(len(a_vgg)):
  d_ap = l1(a_vgg[i], p_vgg[i].detach())
  if not ab:
      d_an = l1(a_vgg[i], n_vgg[i].detach())
      contrastive = d_ap / (d_an + 1e-7)
  else:
      contrastive = d_ap

  loss += contrastive
  print(loss)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.models.resnet import resnet50

class Model(nn.Module):
    def __init__(self, feature_dim=128):
        super(Model, self).__init__()

        self.f = []
        for name, module in resnet50().named_children():
            if name == 'conv1':
                module = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
            if not isinstance(module, nn.Linear) and not isinstance(module, nn.MaxPool2d):
                self.f.append(module)
        # encoder
        self.f = nn.Sequential(*self.f)
        # projection head
        self.g = nn.Linear(2048, 512, bias=False)
        self.g1 = nn.Linear(512, feature_dim, bias=False)

    def forward(self, x):
        x = self.f(x)
        print(x.shape)
        feature = torch.flatten(x, start_dim=1)
        print(feature.shape)
        out = self.g(feature)
        out = self.g1(out)
        return F.normalize(feature, dim=-1), F.normalize(out, dim=-1)

net = Model(feature_dim=128)

a = torch.rand(1, 3, 256, 256)
feature_1, out_1 = net(a)
print(feature_1.shape)
print(out_1.shape)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
def featur(x):
    a1 = torch.rand (1,64,256,256)
    a2 = torch.rand (1,128,128,128)
    a3 = torch.rand (1,256,64,64)
    return [a1, a2, a3]

class ContrastLoss(nn.Module):
    def __init__(self, ablation=False):

        super(ContrastLoss, self).__init__()
        #self.pdist = nn.PairwiseDistance(p=2)
        self.l1 = nn.L1Loss()
        #self.weights = [1.0/32, 1.0/16, 1.0/8]
        self.ab = ablation

    def forward(self, a, p, n):
        a_vgg, p_vgg, n_vgg = featur(a), featur(p), featur(n)

        loss = 0
        tau = 0.07

        d_ap, d_an = 0, 0
        for i in range(len(a_vgg)):

            d_ap = torch.exp(torch.pow((a_vgg[i] - p_vgg[i].detach()),2).mean()/tau)
            d_an = torch.exp(torch.pow((a_vgg[i] - n_vgg[i].detach()),2).mean()/tau)
            contrastive = - torch.log(d_ap / (d_an + d_ap))

            loss += contrastive

        return loss

ctloss = ContrastLoss()
a = torch.rand(1,3,256,256)
b = torch.rand(1,3,256,256)
c = torch.rand(1,3,256,256)
loss = ctloss(a,b,c)
print(loss)

tensor(2.0776)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
def featur(x):
    a1 = torch.rand (1,64,256,256)
    a2 = torch.rand (1,128,128,128)
    a3 = torch.rand (1,256,64,64)
    return [a1, a2, a3]

class ContrastLoss(nn.Module):
    def __init__(self, ablation=False):

        super(ContrastLoss, self).__init__()
        #self.pdist = nn.PairwiseDistance(p=2)
        self.l1 = nn.L1Loss()
        #self.weights = [1.0/32, 1.0/16, 1.0/8]
        self.ab = ablation

    def forward(self, a, p, n):
        a_vgg, p_vgg, n_vgg = featur(a), featur(p), featur(n)

        loss = 0

        d_ap, d_an = 0, 0
        for i in range(len(a_vgg)):

            d_ap = torch.exp(self.l1(a_vgg[i], p_vgg[i].detach()))
            d_an = torch.exp(self.l1(a_vgg[i], n_vgg[i].detach()))
            contrastive = - torch.log(d_ap / (d_an + d_ap))
            aa = self.l1(a_vgg[i], p_vgg[i].detach())

            loss += contrastive

        return loss

ctloss = ContrastLoss()
a = torch.rand(1,3,256,256)
b = torch.rand(1,3,256,256)
c = torch.rand(1,3,256,256)
loss = ctloss(a,b,c)
print(loss)

In [ ]:
# Gamma, Beta block
if self.light:
    FC = [nn.Linear(ngf * mult, ngf * mult, bias=False),
          nn.ReLU(True),
          nn.Linear(ngf * mult, ngf * mult, bias=False),
          nn.ReLU(True)]
else:
    FC = [nn.Linear(img_size // mult * img_size // mult * ngf * mult, ngf * mult, bias=False),
          nn.ReLU(True),
          nn.Linear(ngf * mult, ngf * mult, bias=False),
          nn.ReLU(True)]
self.gamma = nn.Linear(ngf * mult, ngf * mult, bias=False)
self.beta = nn.Linear(ngf * mult, ngf * mult, bias=False)


if self.light:
    x_ = torch.nn.functional.adaptive_avg_pool2d(x, 1)
    x_ = self.FC(x_.view(x_.shape[0], -1))
else:
    x_ = self.FC(x.view(x.shape[0], -1))
gamma, beta = self.gamma(x_), self.beta(x_)


for i in range(self.n_blocks):
            x = getattr(self, 'UpBlock1_' + str(i+1))(x, gamma, beta)